# Part 5: Bitsream generation

## Load MNIST dataset from tf.keras

In [1]:
import tensorflow as tf
import numpy as np
#(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
#x_train = np.expand_dims(x_train, -1)
#x_test = np.expand_dims(x_test, -1)
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

2021-11-28 08:59:52.644039: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:/usr/lib:/usr/lib/x86_64-linux-gnu:/usr/local/lib:/opt/vitis_ai/conda/envs/vitis-ai-tensorflow/lib
2021-11-28 08:59:52.644060: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
print(x_train.shape, x_test.shape,y_train.shape, y_test.shape)

(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 10) (10000, 10)


## We save the test dataset to files so that we can use them on the Pynq card later.

In [3]:
np.save('y_test.npy', y_test)
np.save('x_test.npy', x_test)

## Check the reports
Print out the reports generated by Vivado HLS. Pay attention to the Utilization Estimates' section in particular this time.

In [4]:
import hls4ml
hls4ml.report.read_vivado_report('cifar10-hls-test4')

/workspace/home/ymli/cifar_10_hls4ml/hls4ml/converters/__init__.py:16: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!")
/workspace/home/ymli/cifar_10_hls4ml/hls4ml/converters/__init__.py:23: UserWarning: WARNING: ONNX converter is not enabled!
  warnings.warn("WARNING: ONNX converter is not enabled!")


Found 1 solution(s) in cifar10-hls-test4/myproject_cifar10_cnn4_prj.
Reports for solution "solution1":

C simulation report not found.
SYNTHESIS REPORT:
== Vivado HLS Report for 'myproject_cifar10_cnn4_axi'
* Date:           Sun Nov 28 08:42:30 2021

* Version:        2019.2 (Build 2704478 on Wed Nov 06 22:10:23 MST 2019)
* Project:        myproject_cifar10_cnn4_prj
* Solution:       solution1
* Product family: zynquplus
* Target device:  xczu7ev-ffvc1156-2-e


== Performance Estimates
+ Timing: 
    * Summary: 
    +--------+---------+----------+------------+
    |  Clock |  Target | Estimated| Uncertainty|
    +--------+---------+----------+------------+
    |ap_clk  | 5.00 ns | 4.349 ns |   0.62 ns  |
    +--------+---------+----------+------------+

+ Latency: 
    * Summary: 
    +---------+---------+-----------+-----------+-------+-------+----------+
    |  Latency (cycles) |   Latency (absolute)  |    Interval   | Pipeline |
    |   min   |   max   |    min    |    max    |  min

In [ ]:
hls4ml.report.read_vivado_report('mnist-hls-test2')

In [ ]:
hls4ml.report.read_vivado_report('mnist-hls-test2_reusefactor_8')

In [ ]:
hls4ml.report.read_vivado_report('mnist-hls-test3')

In [ ]:
hls4ml.report.read_vivado_report('mnist-hls-test4')

## Choose the project for making a bitstream file.

In [5]:
prjdir='cifar10-hls-test4'

## Generate bitstream

In [6]:
import os
os.environ['PATH'] = '/workspace/home/Xilinx/Vivado/2019.2/bin:' + os.environ['PATH']
curr_dir = os.getcwd()
os.chdir(prjdir)
try:
    os.system('vivado -mode batch -source design.tcl')
except:
    print("Something went wrong, check the Vivado logs")
os.chdir(curr_dir)    


****** Vivado v2019.2 (64-bit)
  **** SW Build 2708876 on Wed Nov  6 21:39:14 MST 2019
  **** IP Build 2700528 on Thu Nov  7 00:09:20 MST 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source design.tcl
# set tcldir [file dirname [info script]]
# source [file join $tcldir project.tcl]
## variable myproject
## set myproject "myproject_cifar10_cnn4"
## set bit_width_hls_output 32
## set bit_width_hls_input 32
# create_project project_1 ${myproject}_vivado_accelerator -part xczu7ev-ffvc1156-2-e -force
create_project: Time (s): cpu = 00:00:04 ; elapsed = 00:00:29 . Memory (MB): peak = 1512.414 ; gain = 19.016 ; free physical = 9305 ; free virtual = 168383
# set_property board_part xilinx.com:zcu104:part0:1.0 [current_project]
# set_property  ip_repo_paths  ${myproject}_prj [current_project]
# update_ip_catalog
INFO: [IP_Flow 19-234] Refreshing IP repositories
INFO: [IP_Flow 19-1700] Loaded user IP repository '/workspace/home/ymli/cifar_10_hls4ml/cifar10-hls-test4/mypro

# group_bd_cells hier_0 [get_bd_cells axi_dma_0] [get_bd_cells ${myproject}_axi_0]
# make_wrapper -files [get_files ./${myproject}_vivado_accelerator/project_1.srcs/sources_1/bd/design_1/design_1.bd] -top
INFO: [PSU-1]  DP_AUDIO clock source: RPLL is also being used by other peripheral clocks. Their outputs may get impacted if any driver changes DP_AUDIO PLL source to support runtime audio change 
INFO: [PSU-1]  DP_AUDIO clock source: RPLL is also being used by other peripheral clocks. Their outputs may get impacted if any driver changes DP_AUDIO PLL source to support runtime audio change 
INFO: [PSU-1]  DP_AUDIO clock source: RPLL is also being used by other peripheral clocks. Their outputs may get impacted if any driver changes DP_AUDIO PLL source to support runtime audio change 
INFO: [PSU-1]  DP_AUDIO clock source: RPLL is also being used by other peripheral clocks. Their outputs may get impacted if any driver changes DP_AUDIO PLL source to support runtime audio change 
xit::source

INFO: [BD 41-1029] Generation completed for the IP Integrator block hier_0/myproject_cifar10_cnn4_axi_0 .
Exporting to file /workspace/home/ymli/cifar_10_hls4ml/cifar10-hls-test4/myproject_cifar10_cnn4_vivado_accelerator/project_1.srcs/sources_1/bd/design_1/hw_handoff/design_1.hwh
Generated Block Design Tcl file /workspace/home/ymli/cifar_10_hls4ml/cifar10-hls-test4/myproject_cifar10_cnn4_vivado_accelerator/project_1.srcs/sources_1/bd/design_1/hw_handoff/design_1_bd.tcl
Generated Hardware Definition File /workspace/home/ymli/cifar_10_hls4ml/cifar10-hls-test4/myproject_cifar10_cnn4_vivado_accelerator/project_1.srcs/sources_1/bd/design_1/synth/design_1.hwdef
INFO: [IP_Flow 19-5642] Done with IP cache export for multiple IPs
[Sun Nov 28 09:10:17 2021] Launched design_1_rst_ps8_0_100M_0_synth_1, design_1_axi_smc_0_synth_1, design_1_auto_pc_0_synth_1, design_1_auto_ds_1_synth_1, design_1_auto_pc_1_synth_1, design_1_myproject_cifar10_cnn4_axi_0_0_synth_1, design_1_axi_dma_0_0_synth_1, design

Parsing XDC File [/workspace/home/ymli/cifar_10_hls4ml/cifar10-hls-test4/myproject_cifar10_cnn4_vivado_accelerator/project_1.srcs/sources_1/bd/design_1/ip/design_1_zynq_ultra_ps_e_0_0/design_1_zynq_ultra_ps_e_0_0.xdc] for cell 'design_1_i/zynq_ultra_ps_e_0/inst'
create_clock: Time (s): cpu = 00:00:05 ; elapsed = 00:00:10 . Memory (MB): peak = 2793.891 ; gain = 21.781 ; free physical = 12676 ; free virtual = 166587
Finished Parsing XDC File [/workspace/home/ymli/cifar_10_hls4ml/cifar10-hls-test4/myproject_cifar10_cnn4_vivado_accelerator/project_1.srcs/sources_1/bd/design_1/ip/design_1_zynq_ultra_ps_e_0_0/design_1_zynq_ultra_ps_e_0_0.xdc] for cell 'design_1_i/zynq_ultra_ps_e_0/inst'
Parsing XDC File [/workspace/home/ymli/cifar_10_hls4ml/cifar10-hls-test4/myproject_cifar10_cnn4_vivado_accelerator/project_1.srcs/sources_1/bd/design_1/ip/design_1_rst_ps8_0_100M_0/design_1_rst_ps8_0_100M_0_board.xdc] for cell 'design_1_i/rst_ps8_0_100M/U0'
Finished Parsing XDC File [/workspace/home/ymli/cifa

Sourcing Tcl File [/workspace/home/Xilinx/Vivado/2019.2/data/ip/xpm/xpm_memory/tcl/xpm_memory_xdc.tcl] for cell 'design_1_i/axi_smc/inst/m00_nodes/m00_w_node/inst/inst_si_handler/gen_si_handler.gen_request_fifos.gen_req_fifo[1].inst_req_fifo/gen_xpm_memory_fifo.inst_fifo/gen_mem_rep[0].inst_xpm_memory'
Finished Sourcing Tcl File [/workspace/home/Xilinx/Vivado/2019.2/data/ip/xpm/xpm_memory/tcl/xpm_memory_xdc.tcl] for cell 'design_1_i/axi_smc/inst/m00_nodes/m00_w_node/inst/inst_si_handler/gen_si_handler.gen_request_fifos.gen_req_fifo[1].inst_req_fifo/gen_xpm_memory_fifo.inst_fifo/gen_mem_rep[0].inst_xpm_memory'
Sourcing Tcl File [/workspace/home/Xilinx/Vivado/2019.2/data/ip/xpm/xpm_memory/tcl/xpm_memory_xdc.tcl] for cell 'design_1_i/axi_smc/inst/m00_nodes/m00_w_node/inst/inst_mi_handler/gen_normal_area.inst_fifo_node_payld/gen_xpm_memory_fifo.inst_fifo/gen_mem_rep[0].inst_xpm_memory'
Finished Sourcing Tcl File [/workspace/home/Xilinx/Vivado/2019.2/data/ip/xpm/xpm_memory/tcl/xpm_memory_xd

...


Phase 2.2.1 Physical Synthesis In Placer
INFO: [Physopt 32-1018] Found 3983 candidate LUT instances to create LUTNM shape
INFO: [Physopt 32-775] End 1 Pass. Optimized 1759 nets or cells. Created 0 new cell, deleted 1759 existing cells and moved 0 existing cell
INFO: [Physopt 32-65] No nets found for high-fanout optimization.
INFO: [Physopt 32-232] Optimized 0 net. Created 0 new instance.
INFO: [Physopt 32-775] End 1 Pass. Optimized 0 net or cell. Created 0 new cell, deleted 0 existing cell and moved 0 existing cell
INFO: [Physopt 32-670] No setup violation found.  DSP Register Optimization was not performed.
INFO: [Physopt 32-670] No setup violation found.  Shift Register to Pipeline Optimization was not performed.
INFO: [Physopt 32-670] No setup violation found.  Shift Register Optimization was not performed.
INFO: [Physopt 32-670] No setup violation found.  BRAM Register Optimization was not performed.
INFO: [Physopt 32-949] No candidate nets found for HD net replication
INFO: [Phys

Netlist sorting complete. Time (s): cpu = 00:00:00.56 ; elapsed = 00:00:00.57 . Memory (MB): peak = 5639.500 ; gain = 0.000 ; free physical = 10574 ; free virtual = 164524

Phase 4.3 Placer Reporting
Phase 4.3 Placer Reporting | Checksum: 2168e02ae

Time (s): cpu = 00:07:55 ; elapsed = 00:03:36 . Memory (MB): peak = 5639.500 ; gain = 0.000 ; free physical = 10582 ; free virtual = 164532

Phase 4.4 Final Placement Cleanup
Netlist sorting complete. Time (s): cpu = 00:00:00.02 ; elapsed = 00:00:00.02 . Memory (MB): peak = 5639.500 ; gain = 0.000 ; free physical = 10583 ; free virtual = 164533
Phase 4.4 Final Placement Cleanup | Checksum: 2b9b59157

Time (s): cpu = 00:07:56 ; elapsed = 00:03:36 . Memory (MB): peak = 5639.500 ; gain = 0.000 ; free physical = 10585 ; free virtual = 164534
Phase 4 Post Placement Optimization and Clean-Up | Checksum: 2b9b59157

Time (s): cpu = 00:07:56 ; elapsed = 00:03:37 . Memory (MB): peak = 5639.500 ; gain = 0.000 ; free physical = 10586 ; free virtual = 1

 Number of Nodes with overlaps = 1
 Number of Nodes with overlaps = 0
INFO: [Route 35-416] Intermediate Timing Summary | WNS=1.251  | TNS=0.000  | WHS=-0.022 | THS=-0.391 |

Phase 4.1 Global Iteration 0 | Checksum: 1b9fc5c75

Time (s): cpu = 00:10:04 ; elapsed = 00:05:41 . Memory (MB): peak = 5690.012 ; gain = 50.512 ; free physical = 10057 ; free virtual = 164115

Phase 4.2 Global Iteration 1
 Number of Nodes with overlaps = 0
INFO: [Route 35-416] Intermediate Timing Summary | WNS=1.251  | TNS=0.000  | WHS=N/A    | THS=N/A    |

Phase 4.2 Global Iteration 1 | Checksum: 1a6c519bf

Time (s): cpu = 00:10:17 ; elapsed = 00:05:48 . Memory (MB): peak = 5690.012 ; gain = 50.512 ; free physical = 10067 ; free virtual = 164119
Phase 4 Rip-up And Reroute | Checksum: 1a6c519bf

Time (s): cpu = 00:10:17 ; elapsed = 00:05:48 . Memory (MB): peak = 5690.012 ; gain = 50.512 ; free physical = 10066 ; free virtual = 164118

Phase 5 Delay and Skew Optimization

Phase 5.1 Delay CleanUp

Phase 5.1.1 Updat

report_clock_utilization: Time (s): cpu = 00:00:08 ; elapsed = 00:00:20 . Memory (MB): peak = 5806.039 ; gain = 0.000 ; free physical = 9816 ; free virtual = 163945
INFO: [runtcl-4] Executing : report_bus_skew -warn_on_violation -file design_1_wrapper_bus_skew_routed.rpt -pb design_1_wrapper_bus_skew_routed.pb -rpx design_1_wrapper_bus_skew_routed.rpx
INFO: [Timing 38-91] UpdateTimingParams: Speed grade: -2, Temperature grade: E, Delay Type: min_max.
INFO: [Timing 38-191] Multithreading enabled for timing update using a maximum of 8 CPUs
report_bus_skew: Time (s): cpu = 00:00:00.09 ; elapsed = 00:00:12 . Memory (MB): peak = 5806.039 ; gain = 0.000 ; free physical = 9782 ; free virtual = 163909
INFO: [Memdata 28-167] Found XPM memory block design_1_i/hier_0/axi_dma_0/U0/I_PRMRY_DATAMOVER/GEN_S2MM_FULL.I_S2MM_FULL_WRAPPER/GEN_ENABLE_INDET_BTT_SF.I_INDET_BTT/I_XD_FIFO/NON_BLK_MEM.I_SYNC_FIFOGEN_FIFO/xpm_fifo_instance.xpm_fifo_sync_inst/xpm_fifo_base_inst/gen_sdpram.xpm_memory_base_inst wi

Loading data files...
Loading site data...
Loading route data...
Processing options...
Creating bitmap...
Creating bitstream...
Writing bitstream ./design_1_wrapper.bit...
INFO: [Vivado 12-1842] Bitgen Completed Successfully.
INFO: [Project 1-120] WebTalk data collection is mandatory when using a WebPACK part without a full Vivado license. To see the specific WebTalk data collected for your design, open the usage_statistics_webtalk.html or usage_statistics_webtalk.xml file in the implementation directory.
INFO: [Common 17-186] '/workspace/home/ymli/cifar_10_hls4ml/cifar10-hls-test4/myproject_cifar10_cnn4_vivado_accelerator/project_1.runs/impl_1/usage_statistics_webtalk.xml' has been successfully sent to Xilinx on Sun Nov 28 10:27:15 2021. For additional details about this file, please refer to the WebTalk help file at /workspace/home/Xilinx/Vivado/2019.2/doc/webtalk_introduction.html.
INFO: [Common 17-83] Releasing license: Implementation
163 Infos, 174 Warnings, 0 Critical Warnings an